## Overview

#### written on 10/02/24

#### ~~Following tasks had all been accomplished.~~

add profileIconId to account
add item0 to match_account

these two fields turned out to be in my plan; need to go over the records and fetch it again.

~~add new fields~~
~~instantly allow null for addition.~~
run the code to fill up.
set null constraint again.

account 
- profileIconId (smallint)

match_account
- item0 (smallint)

## Import

In [1]:
import time
import importlib
import api_caller
import db_connector

## API call setting

In [2]:
# Reload the module to ensure the latest changes are reflected
importlib.reload(api_caller)

caller = api_caller.APICaller()

# Test your function call
# result = caller.get_account_from_riot_id("NoMatterWhoUR", "NA1")
# print(result)

## DB connection setting

In [3]:
# Reload the module to ensure the latest changes are reflected
importlib.reload(db_connector)

connector = db_connector.DBConnector()

## Query preparation

In [4]:
# accounts

'''
original query:
INSERT INTO accounts (puuid)
VALUES (%s)
ON CONFLICT (puuid)
DO UPDATE SET icon_id = %s
'''

accounts_query = {
    "INSERT": "INTO accounts",
    "ON CONFLICT": "(puuid)",
    "DO UPDATE SET": ""
}

accounts_col = ["puuid"]
accounts_update_col = ["icon_id"]

In [5]:
# match_account

'''
original query:
INSERT INTO match_account (match_id, puuid)
VALUES (%s, %s)
ON CONFLICT (match_id, puuid)
DO UPDATE SET item_0 = %s
'''

match_account_query = {
    "INSERT": "INTO match_account",
    "ON CONFLICT": "(match_id, puuid)",
    "DO UPDATE SET": ""
}

match_account_col = ["match_id", "puuid"]
match_account_update_col = ["item_0"]

## Main code

In [6]:
# only collect matches that have not gotten item_0 column yet.
# "select match_id from matches where  IS NOT NULL"
query = {
    "SELECT": "match_id",
    "FROM": "matches",
    "WHERE": "match_id IN (SELECT match_id FROM match_account WHERE item_0 IS NULL)" 
}
matches = connector.select_query(query)
# call 100 matches at a time
print(len(matches))
# print(matches[:10])

4269


In [7]:
for idx, match in enumerate(matches):
    # print(idx)
    # each match is a tuple with a single element for match list from DB
    data = caller.get_match_from_match_id(match[0])

    for i in range(10):

        accounts_val = (data["info"]["participants"][i]["puuid"], 
                        data["info"]["participants"][i]["profileIcon"]
        )
            
        match_account_val = (data["metadata"]["matchId"],
                             data["info"]["participants"][i]["puuid"], 
                             data["info"]["participants"][i]["item0"]
        )
        connector.insert_query(accounts_query, accounts_col, accounts_val, accounts_update_col)
        connector.insert_query(match_account_query, match_account_col, match_account_val, match_account_update_col)
    if (idx + 1) % 10 == 0:
        print(idx + 1, " matches have been updated")

10  matches have been updated
20  matches have been updated
30  matches have been updated
40  matches have been updated
50  matches have been updated
60  matches have been updated
70  matches have been updated
80  matches have been updated
90  matches have been updated
have called 100 apis: 2 min break starts
100  matches have been updated
110  matches have been updated
120  matches have been updated
130  matches have been updated
140  matches have been updated
150  matches have been updated
160  matches have been updated
170  matches have been updated
180  matches have been updated
190  matches have been updated
have called 200 apis: 2 min break starts
200  matches have been updated
210  matches have been updated
220  matches have been updated
230  matches have been updated
240  matches have been updated
250  matches have been updated
260  matches have been updated
270  matches have been updated
280  matches have been updated
290  matches have been updated
have called 300 apis: 2 min 

In [8]:
print("commited and ended")
connector.commit()
connector.end()

commited and ended
